In [1]:
import numpy as np
from scipy.interpolate import interp1d as interp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from src.thunderflask import thunderflask
from src.bacteria import strain
from src.ffgen import ffgen
from src.codonTable import codonTable
from src.codonUtils import utils
from src.bacteria import strain
import pickle
from src.codonOptimizer import tableOptimizer
from queue import Queue as queue
from random import choice
from matplotlib.colors import LinearSegmentedColormap as LSC
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cProfile
import copy
import random

In [2]:
rc = utils.standardTable
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
        
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop
choices = ['UUU','UAC','UGG','CUC','CCA','CAG','CGU','AUG','ACU','AAA','AGC','GUA','GCC','GAU','GGA']
choices2 = []
for i in range (len(blockArray)):
    choices2.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in choices:
            blockArray[i][key] = '*'
reductionist15 = {}
reductionist16 = {}
reductionist17 = {}
reductionist18 = {}
reductionist19 = {}
reductionist20 = {}
for i in range (len(blockArray)):
    reductionist15.update(blockArray[i])
reductionist16.update(reductionist15)
reductionist16['AAU'] = 'N'
reductionist17.update(reductionist16)  
reductionist17['GAA'] = 'E'
reductionist18.update(reductionist17) 
reductionist18['AUC'] = 'I'
reductionist19.update(reductionist18) 
reductionist19['CAU'] ='H'
reductionist20.update(reductionist19)
reductionist20['UGC'] = 'C'

In [3]:
reductionist20Table = codonTable(reductionist20)
reductionist19Table = codonTable(reductionist19)
reductionist18Table = codonTable(reductionist18)
reductionist17Table = codonTable(reductionist17)
reductionist16Table = codonTable(reductionist16)
reductionist15Table = codonTable(reductionist15)

In [4]:
reductionist15Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : *  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : *  CGU : R
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : P  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : *  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : S
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : V  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : *  GAG : *  GGG : *

In [5]:
reductionist16Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : *  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : *  CGU : R
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : P  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : N  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : S
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : V  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : *  GAG : *  GGG : *

In [6]:
reductionist17Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : *  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : *  CGU : R
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : P  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : N  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : S
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : V  GCA : *  GAA : E  GGA : G
  G  GUG : *  GCG : *  GAG : *  GGG : *

In [7]:
reductionist18Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : *  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : *  CGU : R
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : P  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : N  AGU : *
  C  AUC : I  ACC : *  AAC : *  AGC : S
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : V  GCA : *  GAA : E  GGA : G
  G  GUG : *  GCG : *  GAG : *  GGG : *

In [8]:
reductionist19Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : *  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : R
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : P  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : N  AGU : *
  C  AUC : I  ACC : *  AAC : *  AGC : S
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : V  GCA : *  GAA : E  GGA : G
  G  GUG : *  GCG : *  GAG : *  GGG : *

In [9]:
reductionist20Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : C
  A  UUA : *  UCA : *  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : R
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : P  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : N  AGU : *
  C  AUC : I  ACC : *  AAC : *  AGC : S
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : V  GCA : *  GAA : E  GGA : G
  G  GUG : *  GCG : *  GAG : *  GGG : *